In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import sys
from pathlib import Path
from pprint import pprint
sys.path.append(str(Path.cwd().parent))

from pathlib import Path
import pyrootutils

import torch.nn as nn
import hydra

import numpy as np
import torch
from omegaconf import  OmegaConf
from pytorch_lightning import LightningModule

from tqdm import tqdm
import pandas as pd
import src.utils.default as utils
import SimpleITK as sitk
sitk.ProcessObject_SetGlobalWarningDisplay(False)
from src.data.bvisa_dm import CS_Dataset

import SimpleITK as sitk

from src.data.bvisa_dm import CS_Dataset

import matplotlib.pyplot as plt

from monai.metrics import compute_dice, compute_hausdorff_distance, compute_iou

# Load segmentation model

In [2]:
CHKP = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/logs_finetuning/CS1x_noSST_tverskyLoss_monaBasicUnet-frozenEncoder/runs/2023-04-12_14-11-50/checkpoints/epoch-048-Esubj-0.4790.ckpt')

out_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa_results/nobackup')

exp_name = CHKP.parent.parent.parent.parent.name
out_path = out_path / exp_name
out_path.mkdir(exist_ok=True, parents=True)
cgf_path = CHKP.parent.parent / '.hydra' / 'config.yaml'
finetune_cfg = OmegaConf.load(cgf_path)

segm_model: LightningModule = hydra.utils.instantiate(finetune_cfg.model)
# sst_ds = hydra.utils.instantiate(finetune_cfg.data)
# print(finetune_cfg.data)
segm_model = segm_model.load_from_checkpoint(CHKP).to('cuda')
segm_model = segm_model.eval()


Freezing encoder...
Freezing encoder...


Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
Attribute 'loss_function' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_function'])`.


# Load via validation images

In [3]:
croppadd2same_size =  finetune_cfg.data.dataset_cfg.get('padd2same_size') if finetune_cfg.data.dataset_cfg.get('padd2same_size') else finetune_cfg.data.dataset_cfg.get('croppadd2same_size')

In [4]:
via11DS = CS_Dataset('bvisa', 'skull_stripped',
                     'central_sulcus',
                     dataset_path='/mrhome/vladyslavz/git/central-sulcus-analysis/data/brainvisa',
                     split='validation',
                     preload=False,
                     resample=finetune_cfg.data.dataset_cfg.resample,
                     croppadd2same_size=croppadd2same_size)

In [5]:
experiment_results = []
for idx in tqdm(range(len(via11DS))):
    sample = via11DS[idx]
    target_1hot = torch.nn.functional.one_hot(sample['target'].unsqueeze(0), num_classes=2).permute(0, 4, 1, 2, 3)

    with torch.no_grad():
        out = segm_model.forward(sample['image'].unsqueeze(0).to('cuda'))
    out_bin = torch.argmax(torch.softmax(out, dim=1), dim=1).cpu()
    out_1hot = torch.nn.functional.one_hot(out_bin, num_classes=2).permute(0, 4, 1, 2, 3)

    dice = compute_dice(out_1hot, target_1hot, include_background=False)
    iou = compute_iou(out_1hot, target_1hot, include_background=False)
    hausdorff_distance = compute_hausdorff_distance(out_1hot, target_1hot, include_background=False)

    res = {'caseid': via11DS.caseids[idx],
           'dice': dice.item(),
           'iou': iou.item(),
           'hausdorff_distance': hausdorff_distance.item()
           }
    experiment_results.append(res)
experiment_results = pd.DataFrame(experiment_results)
experiment_results = experiment_results.set_index('caseid')
experiment_results.loc['MEAN'] = experiment_results.mean()
experiment_results.loc['STD'] = experiment_results.std()
experiment_results.to_csv(f'{out_path}/bvisa_metrics.csv')


  0%|          | 0/12 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 992.00 MiB (GPU 0; 11.76 GiB total capacity; 8.51 GiB already allocated; 738.62 MiB free; 10.10 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# plt.imshow(out_hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.imshow(target_1hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.show()



# fig, axs = plt.subplots(1, 1, figsize=(5, 5))
# slice = 120
# axs.imshow(sample['image'][0, slice, :, :], cmap='gray', alpha=0.5)
# axs.imshow(sample['target'][slice, :, :], cmap='gray', alpha=0.5)
# plt.show()

In [ ]:
pprint(experiment_results)

,dice,iou,hausdorff_distance
caseid,,,
eros,0.690456,0.527249,7.211103
icbm320T,0.710104,0.550512,8.246211
moon,0.697471,0.535475,2.236068
ra,0.654259,0.486171,7.280110
s12158,0.662114,0.494895,7.071068
s12258,0.645196,0.476229,10.392305
s12277,0.633105,0.463171,17.146428
shiva,0.638817,0.469310,12.884099
sujet04,0.679640,0.514738,7.615773
